전처리

In [ ]:
import cv2
import numpy as np

In [ ]:
train_data_path = ""
val_datapath = ""

데이터 전처리(파이토치)

In [2]:
train_data_path = "/content/drive/Othercomputers/내 PC/TEST_1/mask_train"
val_data_path = "/content/drive/Othercomputers/내 PC/TEST_1/mask_val"
import torch
from torchvision import datasets, transforms

def data_process(train_data_path,val_data_path,batch_size):
  train = transforms.Compose([
      transforms.Resize((224,224)),#리사이징
      transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),#정규화
      transforms.RandomHorizontalFlip(),# 증강을위한 반전 적용
      transforms.ToTensor()
  ])
  val = transforms.Compose([
      transforms.Resize((224,224)),
      transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
      transforms.ToTensor()
  ])
  #훈련셋 검증셋
  train_data = datasets.ImageFolder(train_data_path,transform=train)
  val_data = datasets.ImageFolder(val_data_path,transform=val)

  #데이터로드 생성
  train_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
  val_loader = torch.utils.data.DataLoader(val_data,batch_size=batch_size,shuffle=False)

  return train_loader,val_loader

데이터전처리(텐서플로우)

In [1]:
import tensorflow as tf

def data_process(train_data_path, val_data_path, batch_size):
    # 훈련 데이터셋 생성: 디렉토리 구조에 따라 라벨 자동 생성
    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        train_data_path,
        image_size=(224, 224),
        batch_size=batch_size,
        shuffle=True
    )

    # 검증 데이터셋 생성
    val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        val_data_path,
        image_size=(224, 224),
        batch_size=batch_size,
        shuffle=False
    )

    # 표준화
    standardization_layer = tf.keras.layers.Normalization(axis=-1)


    train_dataset = train_dataset.map(lambda x, y: (standardization_layer(x), y))
    val_dataset = val_dataset.map(lambda x, y: (standardization_layer(x), y))

    # 데이터 증강: 훈련 데이터에 대해 랜덤 좌우 반전 적용
    augmentation_layer = tf.keras.layers.RandomFlip("horizontal")
    train_dataset = train_dataset.map(lambda x, y: (augmentation_layer(x, training=True), y))

    return train_dataset, val_dataset


# 간단한 cnn 분류모델

In [1]:
train_data_path = "/content/drive/Othercomputers/내 PC/TEST_1/mask_train"
val_data_path = "/content/drive/Othercomputers/내 PC/TEST_1/mask_val"

In [2]:
import tensorflow as tf

# 데이터 전처리 함수: 이미지 로드, 표준화, 데이터 증강 적용
def data_process(train_data_path, val_data_path, batch_size):
    # 원본 훈련 데이터셋 생성 (클래스 이름 추출을 위해 별도로 생성)
    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        train_data_path,
        image_size=(224, 224),
        batch_size=batch_size,
        shuffle=True
    )
    # 원본 검증 데이터셋 생성
    val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        val_data_path,
        image_size=(224, 224),
        batch_size=batch_size,
        shuffle=False
    )

    # 클래스 이름 저장
    class_names = train_dataset.class_names

    # 표준화 레이어: 평균 0, 표준편차 1로 변환
    standardization_layer = tf.keras.layers.Normalization(axis=-1)
    # 표준화 레이어를 훈련 이미지 데이터에 맞춰 학습
    train_images = train_dataset.map(lambda x, y: x)
    standardization_layer.adapt(train_images)

    # 훈련 및 검증 데이터에 표준화 적용
    train_dataset = train_dataset.map(lambda x, y: (standardization_layer(x), y))
    val_dataset = val_dataset.map(lambda x, y: (standardization_layer(x), y))

    # 데이터 증강: 훈련 데이터에 랜덤 좌우 반전 적용
    augmentation_layer = tf.keras.layers.RandomFlip("horizontal")
    train_dataset = train_dataset.map(lambda x, y: (augmentation_layer(x, training=True), y))

    return train_dataset, val_dataset, class_names


In [3]:

# 데이터 경로 및 하이퍼파라미터 설정
train_data_path = "/content/drive/Othercomputers/내 PC/TEST_1/mask_train"
val_data_path = "/content/drive/Othercomputers/내 PC/TEST_1/mask_val"
batch_size = 32

# 전처리된 데이터셋과 클래스 정보를 불러옴
train_dataset, val_dataset, class_names = data_process(train_data_path, val_data_path, batch_size)
num_classes = len(class_names)
print("클래스 이름:", class_names)

# 분류 모델 구성
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# 모델 컴파일: 분류 문제이므로 손실함수로 sparse categorical crossentropy 사용
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
epochs = 15
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs
)

# 모델 저장
model_save_path = "/content/drive/MyDrive/CNN_classification_model_PROTO.h5"
model.save(model_save_path)
print("모델이 저장되었습니다:", model_save_path)


Found 5994 files belonging to 4 classes.
Found 1200 files belonging to 4 classes.
클래스 이름: ['mask_labeled_angry', 'mask_labeled_happy', 'mask_labeled_panic', 'mask_labeled_sadness']
Epoch 1/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 107s 534ms/step - accuracy: 0.2534 - loss: 3.1104 - val_accuracy: 0.2542 - val_loss: 1.3864
Epoch 2/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 34s 179ms/step - accuracy: 0.2417 - loss: 1.3865 - val_accuracy: 0.2583 - val_loss: 1.3862
Epoch 3/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 33s 175ms/step - accuracy: 0.2669 - loss: 1.3853 - val_accuracy: 0.2517 - val_loss: 1.3871
Epoch 4/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 33s 175ms/step - accuracy: 0.2879 - loss: 1.3807 - val_accuracy: 0.2525 - val_loss: 1.3930
Epoch 5/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 33s 174ms/step - accuracy: 0.3227 - loss: 1.3696 - val_accuracy: 0.2842 - val_loss: 1.3985
Epoch 6/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 33s 175ms/step - accuracy: 0.3707 - loss: 1.3213 - val_accuracy: 0.2733 - val_loss: 1.4296
Epoch 7/15
188/188 ━━━━━━━━━━━━━

모델이 저장되었습니다: /content/drive/MyDrive/CNN_classification_model_PROTO.h5


# 데이터 테스트


In [4]:
import tensorflow as tf

# 테스트 데이터 경로
test_data_path = "/content/drive/Othercomputers/내 PC/TEST_1/test/image-20250212T014410Z-001/image"

# 테스트 데이터 불러오기
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_path,
    image_size=(224, 224),
    batch_size=32,
    shuffle=False  # 테스트셋은 순서를 유지하는 것이 좋음
)

# 표준화 함수
def preprocess(image, label):
    # 표준화: 평균과 표준편차를 사용하여 이미지 정규화
    # 평균과 표준편차 계산
    image = tf.cast(image, tf.float32)
    mean = tf.reduce_mean(image)
    stddev = tf.math.reduce_std(image)

    image = (image - mean) / stddev  # 표준화
    return image, label


# 전처리 적용
test_dataset = test_dataset.map(preprocess)

# 모델 평가
test_loss, test_acc = model.evaluate(test_dataset)
print(f"테스트 정확도: {test_acc * 100:.2f}%")


Found 1200 files belonging to 4 classes.
38/38 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.2632 - loss: 2.9634
테스트 정확도: 27.42%
